In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({
    "x0": [1, 2, 3, 4, 5],
    "x1": [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]
})

In [3]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [4]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [5]:
data.to_numpy()

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [6]:
df2 = pd.DataFrame(data.to_numpy(), columns=["one", "two", "three"])

In [7]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [8]:
df3 = data.copy()

In [11]:
df3["strings"] = list('abcde')

In [14]:
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [15]:
df3.to_numpy()

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [16]:
model_cols = ["x0", "x1"]

In [17]:
data.loc[:, model_cols].to_numpy()

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [18]:
data["category"] = pd.Categorical(["a", "b", "a", "a", "b"], categories=["a", "b"])

In [19]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [20]:
dummies = pd.get_dummies(data.category, prefix="category")

In [21]:
data_with_dummies = data.drop("category", axis=1).join(dummies)

In [22]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,True,False
1,2,-0.01,0.0,False,True
2,3,0.25,3.6,True,False
3,4,-4.10,1.3,True,False
4,5,0.00,-2.0,False,True


In [23]:
data = pd.DataFrame({
    "x0": [1, 2, 3, 4, 5],
    "x1": [0.01, -0.01, 0.25, -4.1, 0.],
    "y": [-1.4, 0., 3.6, 1.3, -2.]
})

In [24]:
data

,x0,x1,y
0,1,0.01,-1.4
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [25]:
import patsy

In [26]:
y, x = patsy.dmatrices('y ~ x0 + x1', data)

In [27]:
y

DesignMatrix with shape (5, 1)
     y
  -1.4
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [28]:
x

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [29]:
np.asarray(y)

array([[-1.4],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [30]:
np.asarray(x)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [31]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [33]:
coef, resid, _, _ = np.linalg.lstsq(x, y)

/var/folders/ny/q654wcmn7fq3766flqv2rdv80000gn/T/ipykernel_51327/1226071125.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef,resid,_, _ = np.linalg.lstsq(x, y)


In [34]:
coef

array([[ 0.3930689 ],
       [-0.09924691],
       [-0.26580757]])

In [35]:
coef = pd.Series(coef.squeeze(), index=x.design_info.column_names)

In [36]:
coef

Intercept    0.393069
x0          -0.099247
x1          -0.265808
dtype: float64

In [37]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

In [38]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [39]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [40]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [41]:
new_data = pd.DataFrame({
    "x0": [6, 7, 8, 9],
    "x1": [3.1, -0.5, 0, 2.3],
    "y": [1, 2, 3, 4]
})

In [42]:
new_X = patsy.build_design_matrices([X.design_info], new_data)

In [43]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [44]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [45]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

In [46]:
data

,x0,x1,y
0,1,0.01,-1.4
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [48]:
data = pd.DataFrame({
    "key1": ["a", "a", "b", "b", "a", "b", "a", "b"],
    "key2": [0, 1, 0, 1, 0, 1, 0, 0],
    "v1": list(range(1, 9, 1)),
    "v2": [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

In [49]:
list(range(1, 9, 1))

[1, 2, 3, 4, 5, 6, 7, 8]

In [50]:
list(range(1, 9))

[1, 2, 3, 4, 5, 6, 7, 8]

In [51]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [52]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [53]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [54]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [55]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [56]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [57]:
data['key2'] = data['key2'].map({0: 'zero', 1: "one"})

In [58]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [59]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

In [60]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [62]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [63]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

In [64]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [65]:
rng = np.random.default_rng(seed=12345)

In [68]:
def dnorm(mean, variance, size = 1):
    if isinstance(size, int):
        size = size
    return mean + np.sqrt(variance) * rng.standard_normal(size)

In [69]:
N = 100
X = np.c_[
    dnorm(0, 0.4, size=N),
    dnorm(0, 0.6, size=N),
    dnorm(0, 0.2, size=N),
]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [70]:
X[:5]

array([[-0.90050602, -0.18942958, -1.0278702 ],
       [ 0.79925205, -1.54598388, -0.32739708],
       [-0.55065483, -0.12025429,  0.32935899],
       [-0.16391555,  0.82403985,  0.20827485],
       [-0.04765129, -0.21314698, -0.04824364]])

In [71]:
y[:5]

array([-0.59952668, -0.58845445,  0.18563386, -0.00747657, -0.01537445])

In [72]:
X_model = sm.add_constant(X)

In [73]:
X_model[:5]

array([[ 1.        , -0.90050602, -0.18942958, -1.0278702 ],
       [ 1.        ,  0.79925205, -1.54598388, -0.32739708],
       [ 1.        , -0.55065483, -0.12025429,  0.32935899],
       [ 1.        , -0.16391555,  0.82403985,  0.20827485],
       [ 1.        , -0.04765129, -0.21314698, -0.04824364]])

In [74]:
model = sm.OLS(y,X)

In [75]:
results = model.fit()

In [76]:
results.params

array([0.06681503, 0.26803235, 0.45052319])

In [77]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.469
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.51
Date:                Mon, 17 Jun 2024   Prob (F-statistic):                    2.66e-13
Time:                        17:13:08   Log-Likelihood:                         -25.611
No. Observations:                 100   AIC:                                      57.22
Df Residuals:                      97   BIC:                                      65.04
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [78]:
data = pd.DataFrame(X, columns=["col0", "col1", "col2"])

In [79]:
data["y"] = y

In [80]:
data[:5]

,col0,col1,col2,y
0,-0.900506,-0.189430,-1.027870,-0.599527
1,0.799252,-1.545984,-0.327397,-0.588454
2,-0.550655,-0.120254,0.329359,0.185634
3,-0.163916,0.824040,0.208275,-0.007477
4,-0.047651,-0.213147,-0.048244,-0.015374


In [81]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()

In [82]:
results.params

Intercept   -0.020799
col0         0.065813
col1         0.268970
col2         0.449419
dtype: float64

In [83]:
results

In [84]:
results.tvalues

Intercept   -0.652501
col0         1.219768
col1         6.312369
col2         6.567428
dtype: float64

In [85]:
results.predict(data[:5])

0   -0.592959
1   -0.531160
2    0.058636
3    0.283658
4   -0.102947
dtype: float64

In [87]:
init_x = 4

values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [88]:
from statsmodels.tsa.ar_model import AutoReg

In [89]:
MAXLAGS = 5

In [90]:
model = AutoReg(values, MAXLAGS)

In [91]:
results = model.fit()

In [92]:
results.params

array([ 0.02346612,  0.8096828 , -0.42865278, -0.03336517,  0.04267874,
       -0.05671529])

In [93]:
train = pd.read_csv("datasets/titanic/train.csv")

In [94]:
test = pd.read_csv("datasets/titanic/test.csv")

In [95]:
train.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [96]:
train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [97]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [98]:
impute_value = train["Age"].median()

In [99]:
train["Age"] = train["Age"].fillna(impute_value)

In [100]:
test["Age"] = test["Age"].fillna(impute_value)

In [101]:
train["IsFemale"] = (train["Sex"] == "female").astype(int)

In [102]:
test["IsFemale"] = (test["Sex"] == "female").astype(int)

In [103]:
predictors = ["Pclass", "IsFemale", "Age"]

In [104]:
X_train = train[predictors].to_numpy()

In [105]:
X_test = test[predictors].to_numpy()

In [106]:
y_train = train["Survived"].to_numpy()

In [107]:
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [108]:
y_train[:5]

array([0, 1, 1, 1, 0])

In [109]:
from sklearn.linear_model import LogisticRegression

In [110]:
model = LogisticRegression()

In [111]:
model.fit(X_train, y_train)

LogisticRegression()

In [112]:
y_predict = model.predict(X_test)

In [113]:
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

In [114]:
from sklearn.linear_model import LogisticRegressionCV

In [115]:
model_cv = LogisticRegressionCV(Cs=10)

In [116]:
model_cv.fit(X_train, y_train)

LogisticRegressionCV()

In [117]:
from sklearn.model_selection import cross_val_score

In [118]:
model = LogisticRegression(C=10)

In [119]:
scores = cross_val_score(model, X_train, y_train, cv=4)

In [120]:
scores

array([0.77578475, 0.79820628, 0.77578475, 0.78828829])